# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902038


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:39,  1.41s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:47,  1.75s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:33,  1.27s/it]

Rendering models:  20%|██        | 6/30 [00:05<00:21,  1.10it/s]

...

Rendering models:  87%|████████▋ | 26/30 [00:10<00:00,  6.04it/s]

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  5.97it/s]

Rendering models:  97%|█████████▋| 29/30 [00:10<00:00,  7.32it/s]

Rendering models: 100%|██████████| 30/30 [00:10<00:00,  6.86it/s]

esazure                               0.000117
Jes8x                                 0.000415
nickoftona                            0.000209
equidad1                              0.000139
not-logged-in-7c33d6a0ca9ac4648bdf    0.060695
Thedunkmasta                          0.000325
3lite                                 0.000911
not-logged-in-ab1612bc18cd7aab85db    0.000216
DavidCastillo1702                     0.000199
not-logged-in-7c0e3b1ae8dc7fbb6b66    0.000418
Jessica_Korkmaz                       0.001699
not-logged-in-76a9c02fe5d62b1c2cae    0.178274
AdamMorse                             0.000112
not-logged-in-c016e89f1952c96c30e6    0.001267
doiknowu12345                         0.001025
pangeli5                              0.000356
not-logged-in-214814e52004b42491a4    0.036448
haydenmanninen                        0.000104
manxkats                              0.001059
peakscience7                          0.077859
Lavadude                              0.002601
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())